In [ ]:
import openai
import pandas as pd
import time
from openai import OpenAI


# API key
OPENAI_API_KEY = "sk-proj-xxx"

client = OpenAI(api_key=OPENAI_API_KEY)



# Load the dataset (take the human-written dataset as an example)
df = pd.read_excel("/content/human_dataset.xlsx")

# This is to classify text using the ChatGPT-4o model
def classify_text(text):
    try:
        system_prompt = "You are a detector and your task is to detect whether the given text is written by a human or a Large Language Model. The human writers are Logistics Management Engineering and Geological Engineering graduate students."
        user_prompt = f"""
        Give/Classify the essay a label from/into "AI-generated", "human-written", or "unclear".\n
        Essay:{text}\n
        Label/Class:
        """

        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0,
            max_tokens=30
        )

        label = response.choices[0].message.content.strip()
        return label

    except Exception as e:
        print(f"Error: {e}")
        return "error"

# Classification
predicted_labels = []
for idx, row in df.iterrows():
    print(f"Classifying row {idx + 1}/{len(df)}...")
    predicted_label = classify_text(row['Essay'])
    print(f"Predicted Label: {predicted_label}")

    predicted_labels.append(predicted_label)
    time.sleep(1)

# Add predictions to the dataframe
df["predicted_label"] = predicted_labels

# Save to Excel
df.to_excel("human-written dataset.xlsx", index=False)
print("✅ Classification complete and saved.")
